In [0]:
# Importing Libraries

In [0]:
d=[]
while(1):
  d.append('1')

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = '/content/drive/My Drive/AppliedAI/Human_activity_Regonition/UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/AppliedAI/Human_activity_Regonition/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/My Drive/AppliedAI/Human_activity_Regonition/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
!ls

drive  __pycache__  sample_data  temp_model.py


In [0]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [0]:
Y_train.shape

(7352, 6)

In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
n_hidden

32

In [0]:
for optimizer in ['rmsprop', 'adam', 'sgd']:

  # Initiliazing the sequential model
  model = Sequential()
  # Configuring the parameters
  model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim),return_sequences = True))
  # Adding a dropout layer
  model.add(Dropout(0.5))
  model.add(LSTM(n_hidden))
  # Adding a dropout layer
  model.add(Dropout(0.5))
  # Adding a dense output layer with sigmoid activation
  model.add(Dense(n_classes, activation='sigmoid'))
  # Compiling the model
  model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

  # Training the model
  model.fit(X_train,
            Y_train,
            batch_size=batch_size,
            validation_data=(X_test, Y_test),epochs=epochs)

  score = model.evaluate(X_test, Y_test)
  scores.append(score)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 63s 9ms/step - loss: 1.2038 - acc: 0.5037 - val_loss: 0.9234 - val_acc: 0.5921
Epoch 2/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.8080 - acc: 0.6310 - val_loss: 0.7691 - val_acc: 0.6084
Epoch 3/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.7376 - acc: 0.6511 - val_loss: 0.7521 - val_acc: 0.6026
Epoch 4/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.6590 - acc: 0.6895 - val_loss: 0.7249 - val_acc: 0.6899
Epoch 5/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.5651 - acc: 0.7719 - val_loss: 0.6315 - val_acc: 0.7316
Epoch 6/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.4828 - acc: 0.8396 - val_loss: 0.5976 - val_acc: 0.8490
Epoch 7/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.3592 - acc: 0.8973 - val_loss: 0.6531 - val_acc: 0.8239
Epoch 8

In [0]:
scores

[[0.532612882990775, 0.8893790295215473],
 [0.5068860703896482, 0.8975229046487954],
 [0.6322883553935098, 0.9043094672548354]]

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 29s 4ms/step - loss: 1.3269 - acc: 0.4332 - val_loss: 1.1863 - val_acc: 0.4700
Epoch 2/30
7352/7352 [==============================] - 28s 4ms/step - loss: 1.0317 - acc: 0.5431 - val_loss: 0.9744 - val_acc: 0.5765
Epoch 3/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.8444 - acc: 0.6351 - val_loss: 0.8123 - val_acc: 0.6132
Epoch 4/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.8148 - acc: 0.6325 - val_loss: 0.8621 - val_acc: 0.5955
Epoch 5/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.7198 - acc: 0.6518 - val_loss: 0.8112 - val_acc: 0.6189
Epoch 6/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.7035 - acc: 0.6498 - val_loss: 0.7743 - val_acc: 0.6230
Epoch 7/30
7352/7352 [====================

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 525        0  ...                   0                 0
SITTING                  1      400  ...                   0                 0
STANDING                 0      103  ...                   0                 2
WALKING                  0        0  ...                   9                52
WALKING_DOWNSTAIRS       0        0  ...                 411                 7
WALKING_UPSTAIRS         0        0  ...                  14               450

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 310us/step


In [0]:
score

[0.3423953423414004, 0.8968442483881914]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30

---


- We can further imporve the performace with Hyperparameter tuning

In [0]:
!pip install hyperas

In [0]:
from hyperopt.hp import *

In [0]:
#exec('from __future__ import absolute_import, division, print_function')
import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

def data(X_train, X_test, Y_train, Y_test):
    
    """Data providing function:
    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    return X_train, Y_train, X_test, Y_test


def create_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(LSTM({{choice([16,32,64,128])}},return_sequences=True, input_shape=(timesteps, input_dim)))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(LSTM({{choice([16,32,64,128])}},return_sequences=True))
    model.add(Dropout({{uniform(0, 1)}}))
        # If we choose 'four', add an additional fourth layer
    if {{choice(['two', 'three'])}} == 'three':
      model.add(LSTM({{choice([16,32,64,128])}},return_sequences=True))
      model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(n_classes, activation='sigmoid'))

    # Compiling the model
    model.compile(loss='categorical_crossentropy',
                optimizer={{choice(['rmsprop', 'adam', 'sgd'])}},
              metrics=['accuracy'])

    result = model.fit(x_train, y_train,
              batch_size={{choice([64, 128])}},
              epochs=10,
              verbose=2,
              validation_split=0.1)
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}



    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """


In [0]:
arg_load_data  = load_data

In [0]:
best_run, best_model = optim.minimize(model=create_model,data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      data_args=(X_train, X_test, Y_train, Y_test),
                                      notebook_name='drive/My Drive/AppliedAI/Human_activity_Regonition/simple_notebook')
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/nbconvert/exporters/exporter.py", line 318, in _preprocess
    nbformat.validate(nbc, relax_add_props=True)
  File "/usr/local/lib/python3.6/dist-packages/nbformat/validator.py", line 277, in validate
    raise error
nbformat.validator.NotebookValidationError: {'pip_warning': {'packages': ['hyperas']}} is not valid under any of the given schemas

Failed validating 'oneOf' in display_data['properties']['data']['patternProperties']['^(?!application/json$)[a-zA-Z0-9]+/[a-zA-Z0-9\\-\\+\\.]+$']:

On instance['cells'][37]['outputs'][1]['data']['application/vnd.colab-display-data+json']:
{'pip_warning': {'packages': ['hyperas']}}

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.6/logging/__init__.py", line 994, in emit
    msg = self.format(record)
  File "/usr/lib/python3.6/logging/__init__.py", line 840, in f

NotebookValidationError: ignored